# Tweet Classification

## Data Set Loading

### DATA Load

In [1]:
import pandas as pd

In [2]:
# Concatenate social spambot tweets
bot_tweets = pd.concat([
    pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/Social_spambots_1.csv', encoding='latin-1'),
    pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/social_spambots_2.csv', encoding='latin-1'),
    pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/social_spambots_3.csv', encoding='latin-1')
]).reset_index(drop=True)

# Read genuine account tweets
clean_tweets = pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/genuine_accounts.csv', encoding='latin-1')


C:\Users\hassa\AppData\Local\Temp\ipykernel_12844\2020695786.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/Social_spambots_1.csv', encoding='latin-1'),
C:\Users\hassa\AppData\Local\Temp\ipykernel_12844\2020695786.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/social_spambots_2.csv', encoding='latin-1'),
C:\Users\hassa\AppData\Local\Temp\ipykernel_12844\2020695786.py:5: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('D:/DNNforBotDetection/DNNforBotDetection/input/cresci-2017.csv/Tweets/social_spambots_3.csv', encoding='latin-1')
C:\Users\hassa\AppData\Local\Temp\ipykernel_12844\2020695786.py:9: DtypeWarning: Columns (0) have mixed t

In [3]:
print(bot_tweets.dtypes)

id                           int64
text                        object
source                      object
user_id                      int64
truncated                  float64
in_reply_to_status_id        int64
in_reply_to_user_id          int64
in_reply_to_screen_name     object
retweeted_status_id          int64
geo                        float64
place                       object
contributors               float64
retweet_count                int64
reply_count                  int64
favorite_count               int64
favorited                  float64
retweeted                  float64
possibly_sensitive         float64
num_hashtags                 int64
num_urls                     int64
num_mentions                 int64
created_at                  object
timestamp                   object
crawled_at                  object
updated                     object
dtype: object


### Data Pre-Processing

In [4]:
import re
import emoji
import pandas as pd


In [5]:
# List of columns to drop
columns_to_drop = ['truncated', 'in_reply_to_status_id', 'in_reply_to_user_id', 'in_reply_to_screen_name',
                   'retweeted_status_id', 'geo', 'place', 'contributors', 'favorited', 'retweeted',
                   'possibly_sensitive', 'created_at', 'timestamp', 'crawled_at', 'updated']

# Drop the columns from the DataFrame
bot_tweets = bot_tweets.drop(columns=columns_to_drop)
clean_tweets = clean_tweets.drop(columns=columns_to_drop)

In [6]:
# Drop 50% of samples to make it computationally feasible

clean_tweets = clean_tweets.drop(clean_tweets.sample(frac=0.5, random_state=42).index)
bot_tweets = bot_tweets.drop(bot_tweets.sample(frac=0.5, random_state=42).index)

In [7]:
bot_tweets.dtypes

id                 int64
text              object
source            object
user_id            int64
retweet_count      int64
reply_count        int64
favorite_count     int64
num_hashtags       int64
num_urls           int64
num_mentions       int64
dtype: object

In [8]:
bot_tweets['BotOrNot'] = 1
clean_tweets['BotOrNot'] = 0

Tweets1_df = pd.concat([bot_tweets, clean_tweets])

Tweets_df = Tweets1_df.sample(frac=1).reset_index(drop=True)

In [9]:
# Remove null samples from the 'text' column
Tweets_df = Tweets_df.dropna(subset=['text'])

In [10]:
def replace_hashtags(tweet):
    return re.sub(r'#\w+', '<hashtag>', tweet)

def replace_urls(tweet):
    return re.sub(r'http\S+|www\S+', '<url>', tweet)

def replace_numbers(tweet):
    return re.sub(r'\d+', '<number>', tweet)

def replace_user_mentions(tweet):
    return re.sub(r'@\w+', '<user>', tweet)

def replace_emojis(tweet):
    emoji_dict = {
        ":)": "<smile>",
        "<3": "<heart>",
        ":D": "<lolface>",
        ":|": "<neutralface>",
        ">:(": "<angryface>"
    }
    for emoticon, tag in emoji_dict.items():
        tweet = tweet.replace(emoticon, tag)
    tweet = emoji.demojize(tweet)
    tweet = re.sub(r':\w+:', '', tweet)
    return tweet


In [11]:
def add_allcaps_tag(word):
    if word.isupper() or re.search(r'(\w)\1{2,}', word):
        return f"{word} <allcaps>"
    return word


In [12]:
def preprocess_tweets(df):
    df['text'] = df['text'].str.lower()
    df['text'] = df['text'].apply(replace_hashtags)
    df['text'] = df['text'].apply(replace_urls)
    df['text'] = df['text'].apply(replace_numbers)
    df['text'] = df['text'].apply(replace_user_mentions)
    df['text'] = df['text'].apply(replace_emojis)
    df['text'] = df['text'].apply(lambda tweet: ' '.join(add_allcaps_tag(word) for word in tweet.split()))
    return df


In [13]:
Final_Tweet_df = preprocess_tweets(Tweets_df)

In [14]:
Final_Tweet_df

,id,text,source,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,BotOrNot
0,533098073870704640,rt <user>: these are the kind of pictures that...,"<a href=""http://twitter.com/download/iphone"" r...",5.495493e+08,16372.0,0.0,0.0,0.0,0.0,1.0,0
1,583206953452748800,holly fuck lauren why are you so cute <url>,"<a href=""http://twitter.com/download/android"" ...",2.251879e+09,0.0,0.0,0.0,0.0,0.0,0.0,0
2,514615231088455680,my wife is such a terrible cook the police mad...,"<a href=""http://tweetadder.com"" rel=""nofollow""...",5.311497e+08,0.0,0.0,0.0,0.0,0.0,0.0,1
3,529397298162184192,"man overboard <hashtag> by <user> ""the most la...","<a href=""http://tweetadder.com"" rel=""nofollow""...",3.709602e+08,0.0,0.0,0.0,1.0,1.0,1.0,1
4,528262761097478144,un giorno comprenderã² come possa provare tant...,"<a href=""http://www.emanueladuccio.it"" rel=""no...",4.663804e+08,1.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
3148243,495233689857843200,"tutto questo amore a parole, senza potersi toc...","<a href=""http://www.giorgiasandri1.com"" rel=""n...",4.664575e+08,0.0,0.0,1.0,0.0,0.0,0.0,1
3148244,410163920482357248,<user> suertuda!,"<a href=""http://twitter.com/#!/download/ipad"" ...",1.536902e+08,0.0,0.0,0.0,0.0,0.0,1.0,0
3148245,549118737971154945,"""yesterday was a thousand years ago... in all ...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",4.946875e+08,0.0,0.0,1.0,0.0,0.0,0.0,0
3148246,569457982724943872,lack of sleep,"<a href=""http://twitter.com/download/iphone"" r...",2.286597e+09,0.0,0.0,0.0,0.0,0.0,0.0,0


### GLove model for creating vector embeddings

In [15]:
import numpy as np
from gensim.models import KeyedVectors

In [16]:
from gensim.models import KeyedVectors

glove_path = "D:/DNNforBotDetection/DNNforBotDetection/input/glove.twitter.27B.200d.txt"  # Path to the downloaded GloVe file

word_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)


In [17]:
def get_tweet_embedding(tokens):
    embeddings = []
    for token in tokens:
        if token in word_vectors:
            embeddings.append(word_vectors[token])
    if embeddings:
        return np.mean(embeddings, axis=0)  # Average the word embeddings to get the tweet embedding
    return np.zeros(word_vectors.vector_size)  # Return a zero vector if no valid word embeddings found


In [18]:
Final_Tweet_df['embedding'] = Final_Tweet_df['text'].apply(get_tweet_embedding)

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Concatenate
# Prepare the data for modeling
X_text = np.stack(Final_Tweet_df['embedding'].values)
X_features = Final_Tweet_df[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions']].values
X = np.concatenate((X_text, X_features), axis=1)
y = Final_Tweet_df['BotOrNot'].values

# LSTM Model
lstm_model = Sequential()
lstm_model.add(LSTM(32, input_shape=(X.shape[1],)))
lstm_model.add(Dense(128, activation='relu'))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dense(1, activation='sigmoid'))

# DNN Model
dnn_model = Sequential()
dnn_model.add(Dense(128, activation='relu', input_shape=(X.shape[1],)))
dnn_model.add(Dense(64, activation='relu'))
dnn_model.add(Dense(1, activation='sigmoid'))

# Concatenate LSTM and features models
combined_model = Sequential()
combined_model.add(Concatenate([lstm_model, dnn_model]))
combined_model.add(Dense(1, activation='sigmoid'))

# Compile the models
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
dnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
combined_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


MemoryError: 

In [ ]:
# Train the model
history = combined_model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

# Retrieve the loss and accuracy
loss = history.history['loss']
accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

# Print the final loss and accuracy
print("Final Loss:", loss[-1])
print("Final Accuracy:", accuracy[-1])

# Print the final validation loss and accuracy
print("Final Validation Loss:", val_loss[-1])
print("Final Validation Accuracy:", val_accuracy[-1])
